## Ejemplo 3.7

#### Realizamos los imports y definimos el fichero .json

In [0]:
%scala
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{col, expr, when, concat, lit}

val file = "/databricks-datasets/learning-spark-v2/blogs.json"

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{col, expr, when, concat, lit}
file: String = /databricks-datasets/learning-spark-v2/blogs.json

#### Creación del esquema y lectura del fichero con dicho esquema

In [0]:
%scala
val schema = StructType(Array(StructField("Id", IntegerType, false),
      StructField("First", StringType, false),
      StructField("Last", StringType, false),
      StructField("Url", StringType, false),
      StructField("Published", StringType, false),
      StructField("Hits", IntegerType, false),
      StructField("Campaigns", ArrayType(StringType), false)))

val blogsDf = spark.read.schema(schema).json(file)

schema: org.apache.spark.sql.types.StructType = StructType(StructField(Id,IntegerType,false),StructField(First,StringType,false),StructField(Last,StringType,false),StructField(Url,StringType,false),StructField(Published,StringType,false),StructField(Hits,IntegerType,false),StructField(Campaigns,ArrayType(StringType,true),false))
blogsDf: org.apache.spark.sql.DataFrame = [Id: int, First: string ... 5 more fields]

#### Mostramos los datos del DataFrame y los datos del esquema de este

In [0]:
%scala
blogsDf.show(false)
println(blogsDf.printSchema())
println(blogsDf.schema)

+---+---------+-------+-----------------+---------+-----+----------------------------+
Id |First |Last |Url |Published|Hits |Campaigns |
+---+---------+-------+-----------------+---------+-----+----------------------------+
1 |Jules |Damji |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn] |
2 |Brooke |Wenig |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn] |
3 |Denny |Lee |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
4 |Tathagata|Das |https://tinyurl.4|5/12/2018|10568|[twitter, FB] |
5 |Matei |Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
6 |Reynold |Xin |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn] |
+---+---------+-------+-----------------+---------+-----+----------------------------+

root
-- Id: integer (nullable = true)
-- First: string (nullable = true)
-- Last: string (nullable = true)
-- Url: string (nullable = true)
-- Published: string (nullable = true)
-- Hits: integer (nullable = true)
-- Campaigns: array (nullable = true)
 |-- element: string (containsNull = true)

()
StructType(StructField(Id,IntegerType,true),StructField(First,StringType,true),StructField(Last,StringType,true),StructField(Url,StringType,true),StructField(Published,StringType,true),StructField(Hits,IntegerType,true),StructField(Campaigns,ArrayType(StringType,true),true))

#### Creamos una vista temporal de los datos

In [0]:
%scala
blogsDf.createOrReplaceTempView("blogs")

#### Creación de DDL a partir del esquema de la tabla

In [0]:
%scala
spark.table("blogs").schema.toDDL

res13: String = Id INT,First STRING,Last STRING,Url STRING,Published STRING,Hits INT,Campaigns ARRAY<STRING>

#### Resultado de los _Hits_ multiplicado por 2

In [0]:
%scala
blogsDf.select(expr("Hits") * 2).show()

+----------+
(Hits * 2)|
+----------+
 9070|
 17816|
 15318|
 21136|
 81156|
 51136|
+----------+

#### Resultado de los _Hits_ sumado a su _Id_

In [0]:
%scala
blogsDf.select(expr("Hits") + expr("Id")).show(false)

+-----------+
(Hits + Id)|
+-----------+
4536 |
8910 |
7662 |
10572 |
40583 |
25574 |
+-----------+

#### Creación de una nueva columna llamada _BigHitters_ la cual añade un valor booleano según si sus _Hits_ son mayor a 10000 o no

In [0]:
%scala
blogsDf.withColumn("Big Hitters", (expr("Hits") > 10000)).show()

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 Id| First| Last| Url|Published| Hits| Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]| false|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]| false|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...| false|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]| true|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...| true|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]| true|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+

#### Creación de una columna nueva con la concatenación del nombre, apellido e _Id_ del usuario, llamado _AuthorsId_

In [0]:
%scala
blogsDf.withColumn("AuthorsId", (concat(expr("First"), expr("Last"), expr("Id")))).select(expr("AuthorsId")).show()

+-------------+
 AuthorsId|
+-------------+
 JulesDamji1|
 BrookeWenig2|
 DennyLee3|
TathagataDas4|
MateiZaharia5|
 ReynoldXin6|
+-------------+